# Review Results
General notebook for reviewing results files

In [1]:
import pandas as pd

In [3]:
experiment_name = "algs-gpu-2-datasets-b"

# get the list of datasets and algs that we expect for this batch
datasets_list_file = "../scripts/DATASETS_B.sh"
algs_list_file = "../scripts/ALGS_GPU_2.sh"

In [4]:
# parse the expected datasets and algs from the scripts

alg_list = []
dataset_list = []

with open(algs_list_file, 'r') as f:
    lines = f.readlines()
    start = False
    for i, l in enumerate(lines):
        if "MODELS_ENVS" in l:
            start = True
            continue
        if start and (")" in l):
            break
        if start:
            alg_list.append(l.split(":")[0].strip())


with open(datasets_list_file, 'r') as f:
    lines = f.readlines()
    start = False
    for i, l in enumerate(lines):
        if "DATASETS" in l:
            start = True
            continue
        if start and (")" in l):
            break
        if start:
            dataset_list.append(l.split("\n")[0].strip())

print(f"found {len(alg_list)} algs in {algs_list_file}:")
for i, a in enumerate(alg_list):
    print(i+1, a)
print()
print(f"found {len(dataset_list)} datasets in {datasets_list_file}: ")
for i, d in enumerate(dataset_list):
    print(i+1, d)



found 10 algs in ../scripts/ALGS_GPU_2.sh:
1 TabTransformer
2 NODE
3 STG
4 NAM
5 DeepFM
6 SAINT
7 DANet
8 rtdl_MLP
9 rtdl_ResNet
10 rtdl_FTTransformer

found 82 datasets in ../scripts/DATASETS_B.sh: 
1 openml__acute-inflammations__10089
2 openml__ada_agnostic__3896
3 openml__aloi__12732
4 openml__analcatdata_boxing1__3540
5 openml__analcatdata_chlamydia__3739
6 openml__anneal__2867
7 openml__artificial-characters__14964
8 openml__audiology__7
9 openml__autos__9
10 openml__bodyfat__5514
11 openml__car-evaluation__146192
12 openml__cardiotocography__9979
13 openml__Census-Income__168340
14 openml__chess__3952
15 openml__chscase_foot__5012
16 openml__cjs__14967
17 openml__cleveland__2285
18 openml__Click_prediction_small__190408
19 openml__colic__25
20 openml__colic__27
21 openml__collins__3567
22 openml__cpu_small__4883
23 openml__dataset_sales__190418
24 openml__dermatology__35
25 openml__EgyptianSkulls__5040
26 openml__elevators__3711
27 openml__fertility__9984
28 openml__fl2000__3566


In [5]:
df = pd.read_csv("../TabSurvey/metadataset.csv")

# filter by experiment name
df = df.loc[df["exp_name"].str.contains(experiment_name)]

In [6]:
print(len(df))

79200


In [7]:
# for each alg-dataset combination, check for results in the metadataset
import itertools

alg_dataset_pairs = list(itertools.product(alg_list, dataset_list))

alg_list_tmp = []
dataset_list_tmp = []
counts_list_tmp = []
# keep track of the number of results for each pair
for alg, dataset in alg_dataset_pairs:
    alg_list_tmp.append(alg)
    dataset_list_tmp.append(dataset)
    counts_list_tmp.append(sum((df["dataset_name"] == dataset) & (df["alg_name"] == alg)))

df_counts = pd.DataFrame(
    {
        "alg": alg_list_tmp,
        "dataset": dataset_list_tmp,
        "count": counts_list_tmp
    }
)


In [8]:
# pairs with fewer than 300 results (300 = complete.)

print("pairs with fewer than 300 results")
df_counts[df_counts["count"] < 300]

pairs with fewer than 300 results


,alg,dataset,count
2,TabTransformer,openml__aloi__12732,70
3,TabTransformer,openml__analcatdata_boxing1__3540,0
4,TabTransformer,openml__analcatdata_chlamydia__3739,0
5,TabTransformer,openml__anneal__2867,220
6,TabTransformer,openml__artificial-characters__14964,0
...,...,...,...
815,rtdl_FTTransformer,openml__visualizing_livestock__3731,0
816,rtdl_FTTransformer,openml__walking-activity__9945,0
817,rtdl_FTTransformer,openml__Wine__190420,0
818,rtdl_FTTransformer,openml__Wisconsin-breast-cancer-cytology-featu...,0


In [9]:
# pairs with fewer than 0 results (complete fail)

print("pairs with 0 results (complete fail)")
df_counts[df_counts["count"] == 0]

pairs with 0 results (complete fail)


,alg,dataset,count
3,TabTransformer,openml__analcatdata_boxing1__3540,0
4,TabTransformer,openml__analcatdata_chlamydia__3739,0
6,TabTransformer,openml__artificial-characters__14964,0
7,TabTransformer,openml__audiology__7,0
10,TabTransformer,openml__car-evaluation__146192,0
...,...,...,...
815,rtdl_FTTransformer,openml__visualizing_livestock__3731,0
816,rtdl_FTTransformer,openml__walking-activity__9945,0
817,rtdl_FTTransformer,openml__Wine__190420,0
818,rtdl_FTTransformer,openml__Wisconsin-breast-cancer-cytology-featu...,0


In [10]:
# for each dataset, what % of algorithms are there at least 200 results?
print(df_counts.groupby("dataset")["count"].apply(lambda x: sum(x > 200) / float(len(alg_list))).sort_values())

dataset
openml__Census-Income__168340                                0.0
openml__walking-activity__9945                               0.0
openml__kropt__2076                                          0.0
openml__aloi__12732                                          0.0
openml__chess__3952                                          0.0
                                                            ... 
openml__meta__4729                                           0.5
openml__labor__4                                             0.5
openml__Wisconsin-breast-cancer-cytology-features__361003    0.5
openml__heart-c__48                                          0.5
openml__cleveland__2285                                      0.5
Name: count, Length: 82, dtype: float64


Several datasets with few/no results. need to look into this.

In [11]:
# for each alg, what % of datasets are there at least 200 results?
print(df_counts.groupby("alg")["count"].apply(lambda x: sum(x > 200) / float(len(dataset_list))).sort_values())

alg
DANet                 0.000000
SAINT                 0.000000
rtdl_FTTransformer    0.000000
rtdl_MLP              0.000000
rtdl_ResNet           0.000000
NAM                   0.512195
NODE                  0.524390
DeepFM                0.597561
TabTransformer        0.621951
STG                   0.865854
Name: count, dtype: float64


Many algs with no results. looks like thesse were not run.